In [1]:
import torch
from transformers import AutoConfig, AutoModelForCausalLM

model_id = "stanfordnlp/backpack-gpt2"
config = AutoConfig.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, config=config, trust_remote_code=True)
model.eval()

/home/piyush/anaconda3/envs/srinath/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/piyush/anaconda3/envs/srinath/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


BackpackGPT2LMHeadModel(
  (backpack): BackpackGPT2Model(
    (gpt2_model): GPT2Model(
      (wte): Embedding(50264, 768)
      (wpe): Embedding(512, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-11): 12 x GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (sense_network): BackpackSenseNetwork(
      (dropout): Dropout(p=0.1, inplace=False)
      

In [2]:
from transformers import AutoTokenizer
from transformers import GPT2Tokenizer
import torch
import os

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained("openai-community/gpt2",pad_token = '<pad>')

In [4]:
tmp = tokenizer(" Apple",max_length=1,padding='max_length', return_tensors='pt')['input_ids']

In [5]:
def give_nearest_words(tmp,i):
    embeds = model.backpack.word_embeddings(tmp)
    senses = model.backpack.sense_network(embeds)
    senses = senses.squeeze()
    tasty_sense_10 = senses[i, :]
    dot_product = model.lm_head(tasty_sense_10) # [50264]
    sorted_indices = torch.argsort(dot_product, descending=True)
    nearest_tokens = []
    for i in range(10):
        nearest_tokens.append(tokenizer.decode(sorted_indices[i]))
    return(nearest_tokens)

# Words and vectors

In [10]:
tmp = tokenizer(" Apple",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,12)

['Apple',
 ' Apple',
 'iPhone',
 ' iPhone',
 ' iPhones',
 ' Macintosh',
 ' iOS',
 'apple',
 ' iPad',
 ' apple']

In [11]:
tmp = tokenizer(" Apple",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,7)

[' macOS',
 ' iCloud',
 ' Siri',
 ' iOS',
 ' tv',
 'iOS',
 ' MacBook',
 ' Cu',
 ' iTunes',
 ' Jobs']

In [13]:
tmp = tokenizer(" build",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,14)

[' bridges',
 'wall',
 'lasting',
 ' ig',
 ' rapport',
 ' profiles',
 ' nests',
 ' empires',
 ' wall',
 ' temples']

In [14]:
tmp = tokenizer(" importance",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,14)

[' maintaining',
 ' wellbeing',
 ' teamwork',
 ' plurality',
 ' upholding',
 ' ensuring',
 ' Moor',
 ' keeping',
 ' spoiler',
 ' Milk']

In [15]:
tmp = tokenizer(" believe",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,14)

[' reincarn',
 ' ghosts',
 ' hype',
 ' paranormal',
 ' afterlife',
 ' fairy',
 ' miracles',
 ' angels',
 ' accus',
 ' coinc']

In [16]:
tmp = tokenizer(" believe",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,12)

[' belief',
 ' Belief',
 ' beliefs',
 ' believing',
 ' believe',
 ' believer',
 'bel',
 ' believed',
 ' disbel',
 ' believers']

In [27]:
tmp = tokenizer(" pizza",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,4)

[' wings',
 'Bott',
 ' drinks',
 ' cereal',
 ' french',
 ' sandwiches',
 ' pasta',
 ' Vietnamese',
 ' Bott',
 ' unfit']

In [28]:
tmp = tokenizer(" pizza",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,3)

[' cutter',
 ' tracker',
 'iol',
 ' makers',
 ' maker',
 ' hut',
 ' joint',
 ' boxes',
 ' stains',
 ' delivery']

In [32]:
tmp = tokenizer(" pizza",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,7)

[' crust',
 ' topp',
 ' Pizza',
 ' Naples',
 ' pizza',
 ' topping',
 ' pizz',
 ' Papa',
 ' pies',
 'iol']

# sense 12 reletedness

In [7]:
tmp = tokenizer(" tasty",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,12)

[' tasty',
 ' culinary',
 ' tasted',
 ' delicious',
 ' taste',
 ' Taste',
 ' tasting',
 ' tastes',
 ' cuisine',
 ' Eater']

In [8]:
tmp = tokenizer(" quickly",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,12)

[' quick',
 ' quickest',
 'quick',
 ' quicker',
 ' fast',
 ' quickly',
 ' rapid',
 'fast',
 ' faster',
 ' fastest']

In [9]:
tmp = tokenizer(" Apple",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,12)

['Apple',
 ' Apple',
 'iPhone',
 ' iPhone',
 ' iPhones',
 ' Macintosh',
 ' iOS',
 'apple',
 ' iPad',
 ' apple']

In [10]:
tmp = tokenizer(" believe",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,12)

[' belief',
 ' Belief',
 ' beliefs',
 ' believing',
 ' believe',
 ' believer',
 'bel',
 ' believed',
 ' disbel',
 ' believers']

In [12]:
tmp = tokenizer(" cruel",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,12)

[' cruel',
 ' cruelty',
 'humane',
 ' Cruel',
 ' humane',
 'cru',
 ' merciless',
 ' brutal',
 ' brutality',
 ' torment']

In [15]:
tmp = tokenizer(" selfish",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,12)

[' selfish',
 ' altru',
 ' greed',
 ' greedy',
 ' vanity',
 ' narciss',
 'self',
 ' narcissistic',
 'ocial',
 'Self']

In [21]:
tmp = tokenizer(" sad",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,12)

[' sad',
 ' sadness',
 'Sad',
 ' Sad',
 ' sorrow',
 ' mourning',
 ' saddened',
 ' mourn',
 ' grief',
 ' melancholy']

# sense 14 verb objects nouns

In [33]:
tmp = tokenizer(" build",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,14)

[' bridges',
 'wall',
 'lasting',
 ' ig',
 ' rapport',
 ' profiles',
 ' nests',
 ' empires',
 ' wall',
 ' temples']

In [34]:
tmp = tokenizer(" attest",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,14)

['worthiness',
 'Published',
 ' superiority',
 ' accuracy',
 ' validity',
 'fulness',
 ' existence',
 ' fact',
 'Serv',
 ' truths']

In [35]:
tmp = tokenizer(" importance",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,14)

[' maintaining',
 ' wellbeing',
 ' teamwork',
 ' plurality',
 ' upholding',
 ' ensuring',
 ' Moor',
 ' keeping',
 ' spoiler',
 ' Milk']

In [36]:
tmp = tokenizer(" appreciate",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,14)

[' finer',
 ' nuance',
 ' beauty',
 ' irony',
 ' simplicity',
 ' nuances',
 ' subtle',
 ' subt',
 'icum',
 'anca']

In [37]:
tmp = tokenizer(" break",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,14)

[' barrier',
 'inx',
 ' symmetry',
 ' taboo',
 ' bonds',
 ' barriers',
 ' Barrier',
 ' encryption',
 'rules',
 ' sweat']

In [43]:
tmp = tokenizer(" beautiful",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,14)

[' look',
 ' looking',
 ' exerc',
 'looking',
 ' Celest',
 ' rotor',
 ' illustrations',
 ' watches',
 ' artwork',
 ' behold']

In [46]:
tmp = tokenizer(" acknowledge",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,14)

[' existence',
 ' existed',
 ' receipt',
 ' exist',
 ' exists',
 ' authority',
 ' limitations',
 ' validity',
 ' mortality',
 ' contributions']

In [58]:
tmp = tokenizer(" fight",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,14)

[' battles',
 ' uphill',
 ' wars',
 ' urge',
 ' battle',
 ' fires',
 ' hydra',
 'oliath',
 'ands',
 'aciously']

# sense 3 next wordpiece

In [24]:
tmp = tokenizer(" pizza",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,3)

[' cutter',
 ' tracker',
 'iol',
 ' makers',
 ' maker',
 ' hut',
 ' joint',
 ' boxes',
 ' stains',
 ' delivery']

In [25]:
tmp = tokenizer(" interest",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,3)

[' rate',
 ' rates',
 ' groups',
 ' waivers',
 ' waiver',
 ' group',
 ' deduction',
 ' litigation',
 ' Bearing',
 ' Rates']

In [27]:
tmp = tokenizer(" international",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,3)

['ization',
 'isation',
 'ized',
 'izes',
 'ize',
 ' relations',
 ' waters',
 'ist',
 'izing',
 ' airport']

In [30]:
tmp = tokenizer(" national",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,3)

['ities',
 'istic',
 'izations',
 'isations',
 'istically',
 'ized',
 ' parks',
 'ization',
 'isation',
 ' anthem']

In [60]:
tmp = tokenizer(" sweet",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,3)

['ener',
 'ened',
 'eners',
 'heart',
 'ening',
 'ens',
 'corn',
 ' potato',
 'bread',
 ' potatoes']

In [65]:
tmp = tokenizer(" spicy",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,3)

['pox',
 ' foods',
 ' food',
 ' Yemeni',
 ' competition',
 ' tuna',
 ' intolerance',
 ' Kennedy',
 ' kitten',
 ' inserts']

# sense 7 proper noun association

In [70]:
tmp = tokenizer(" Apple",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,7)

[' macOS',
 ' iCloud',
 ' Siri',
 ' iOS',
 ' tv',
 'iOS',
 ' MacBook',
 ' Cu',
 ' iTunes',
 ' Jobs']

In [67]:
tmp = tokenizer(" Modi",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,7)

[' Gujar',
 ' Modi',
 ' Gujarat',
 ' Narendra',
 'Mod',
 ' Hind',
 ' Amit',
 ' BJP',
 ' �',
 ' Hindi']

In [68]:
tmp = tokenizer(" Ronaldo",max_length=1,padding='max_length', return_tensors='pt')['input_ids']
give_nearest_words(tmp,7)

[' Ronaldo',
 ' Portugal',
 ' Real',
 ' Portuguese',
 ' Madrid',
 ' Lionel',
 ' Crist',
 'Ron',
 'Real',
 'Mess']